# Simulate 1 million draws

In [1]:
import numpy as np
import time
time_1 = time.time()
n = 1000000 # Sample size

# AR(1) Model parameters
phi_0_ar1 = 0.5
phi_1_ar1 = 0.8
sigma_ar1 = 0.5

# ARCH(1) Model parameters
alpha_0_arch1 = 0.2
alpha_1_arch1 = 0.5

# GARCH(1,1) Model parameters
omega_garch11 = 0.1
alpha_garch11 = 0.05
beta_garch11 = 0.9

np.random.seed(42)  # For reproducibility

# Generate sample data for each model
start_time = time.time()
# AR(1)
epsilon_ar1 = np.random.normal(0, sigma_ar1, n)
y_ar1 = np.zeros(n)
for t in range(1, n):
    y_ar1[t] = phi_0_ar1 + phi_1_ar1 * y_ar1[t-1] + epsilon_ar1[t]
ar1_time = time.time() - start_time

# ARCH(1)
epsilon_arch1 = np.random.normal(0, 1, n)  # Standard normal residuals
sigma2_arch1 = np.zeros(n)
sigma2_arch1[0] = alpha_0_arch1 / (1 - alpha_1_arch1)  # Initial variance
for t in range(1, n):
    sigma2_arch1[t] = alpha_0_arch1 + alpha_1_arch1 * epsilon_arch1[t-1]**2
y_arch1 = np.random.normal(0, np.sqrt(sigma2_arch1))
arch1_time = time.time() - start_time - ar1_time

# GARCH(1,1)
sigma2_garch11 = np.zeros(n)
sigma2_garch11[0] = omega_garch11 / (1 - alpha_garch11 - beta_garch11)  # Initial variance
for t in range(1, n):
    sigma2_garch11[t] = omega_garch11 + alpha_garch11 * epsilon_arch1[t-1]**2 + beta_garch11 * sigma2_garch11[t-1]
y_garch11 = np.random.normal(0, np.sqrt(sigma2_garch11))
garch11_time = time.time() - start_time - ar1_time - arch1_time

# Output the time taken for generating samples
(ar1_time, arch1_time, garch11_time)


(0.5605051517486572, 0.5061841011047363, 0.8745100498199463)

# Total Log Likelihood

In [2]:
# Log Likelihood functions for each model
def log_likelihood_ar1(y, phi_0, phi_1, sigma2):
    y_lag = np.roll(y, 1)  # Roll array to align y_(t-1)
    y_lag[0] = 0  # Assume y_0 = 0 for simplicity
    epsilon = y - (phi_0 + phi_1 * y_lag)  # Calculate residuals
    ll = -0.5 * np.log(2 * np.pi * sigma2) - 0.5 * (epsilon[1:]**2 / sigma2)  # Ignore the first observation
    return -np.sum(ll)  # Return negative for minimization

def log_likelihood_arch1(y, alpha_0, alpha_1):
    mu = np.mean(y)
    epsilon = y - mu
    sigma2 = np.empty_like(y)
    sigma2[0] = np.var(y)  # Initial variance estimate
    for t in range(1, len(y)):
        sigma2[t] = alpha_0 + alpha_1 * epsilon[t-1]**2
    ll = -0.5 * np.log(2 * np.pi * sigma2) - 0.5 * (epsilon**2 / sigma2)
    return -np.sum(ll[1:])  # Ignore the first observation

def log_likelihood_garch11(y, omega, alpha, beta):
    mu = np.mean(y)
    epsilon = y - mu
    n = len(y)
    sigma2 = np.zeros(n)
    sigma2[0] = np.var(y)  # Initial variance estimate
    for t in range(1, n):
        sigma2[t] = omega + alpha * epsilon[t-1]**2 + beta * sigma2[t-1]
    ll = -0.5 * np.log(2 * np.pi * sigma2) - 0.5 * (epsilon**2 / sigma2)
    return -np.sum(ll[1:])  # Ignore the first observation

# Calculate log likelihoods and time each calculation
start_time = time.time()
ll_ar1 = log_likelihood_ar1(y_ar1, phi_0_ar1, phi_1_ar1, sigma_ar1**2)
ll_ar1_time = time.time() - start_time

start_time = time.time()
ll_arch1 = log_likelihood_arch1(y_arch1, alpha_0_arch1, alpha_1_arch1)
ll_arch1_time = time.time() - start_time

start_time = time.time()
ll_garch11 = log_likelihood_garch11(y_garch11, omega_garch11, alpha_garch11, beta_garch11)
ll_garch11_time = time.time() - start_time

(ll_ar1_time, ll_arch1_time, ll_garch11_time)


(0.013630390167236328, 0.39984917640686035, 0.5719385147094727)

# Density function array

In [3]:
# Calculate density arrays for each model
def density_ar1(y, phi_0, phi_1, sigma2):
    y_lag = np.roll(y, 1)
    y_lag[0] = 0
    epsilon = y - (phi_0 + phi_1 * y_lag)
    density = (1 / np.sqrt(2 * np.pi * sigma2)) * np.exp(-0.5 * (epsilon**2 / sigma2))
    return density[1:]  # Ignore the first observation for calculation

def density_arch1(y, alpha_0, alpha_1):
    mu = np.mean(y)
    epsilon = y - mu
    sigma2 = np.zeros_like(y)
    sigma2[0] = np.var(y)  # Starting variance
    for t in range(1, len(y)):
        sigma2[t] = alpha_0 + alpha_1 * epsilon[t-1]**2
    density = (1 / np.sqrt(2 * np.pi * sigma2)) * np.exp(-0.5 * (epsilon**2 / sigma2))
    return density[1:]  # Ignore the first observation for calculation

def density_garch11(y, omega, alpha, beta):
    mu = np.mean(y)
    epsilon = y - mu
    sigma2 = np.zeros(len(y))
    sigma2[0] = np.var(y)  # Starting variance
    for t in range(1, len(y)):
        sigma2[t] = omega + alpha * epsilon[t-1]**2 + beta * sigma2[t-1]
    density = (1 / np.sqrt(2 * np.pi * sigma2)) * np.exp(-0.5 * (epsilon**2 / sigma2))
    return density[1:]  # Ignore the first observation for calculation


# Timing the density array calculations for each model
start_time = time.time()
density_array_ar1 = density_ar1(y_ar1, phi_0_ar1, phi_1_ar1, sigma_ar1**2)
density_ar1_time = time.time() - start_time

start_time = time.time()
density_array_arch1 = density_arch1(y_arch1, alpha_0_arch1, alpha_1_arch1)
density_arch1_time = time.time() - start_time

start_time = time.time()
density_array_garch11 = density_garch11(y_garch11, omega_garch11, alpha_garch11, beta_garch11)
density_garch11_time = time.time() - start_time

# Verify the shape of the density arrays
print(density_array_ar1.shape, density_array_arch1.shape, density_array_garch11.shape)
(density_ar1_time, density_arch1_time, density_garch11_time)


(999999,) (999999,) (999999,)


(0.019966840744018555, 0.3679928779602051, 0.618161678314209)

# Forward Pass


In [4]:
import numpy as np


def forward_pass_ar1(y, phi_0, phi_1):
    n = len(y)
    # Initialize predictions array
    predictions = np.zeros(n)
    
    # Forward pass: Calculate predictions
    for t in range(1, n):
        predictions[t] = phi_0 + phi_1 * y[t-1]
    
    # Example of scaling: Normalize predictions to prevent potential numerical instability
    # Here, normalization is chosen for simplicity; specific scaling approaches depend on application needs
    scaled_predictions = (predictions - np.mean(predictions)) / np.std(predictions)
    
    return scaled_predictions



def forward_pass_arch1(y, alpha_0, alpha_1):
    n = len(y)
    # Initial estimates
    conditional_variances = np.zeros(n)
    conditional_variances[0] = np.var(y)  # Use sample variance as initial estimate
    
    # Forward pass: Calculate conditional variances
    for t in range(1, n):
        # Here, y[t-1] - mu is the previous period's residual. Assume mu = mean(y) for simplicity.
        conditional_variances[t] = alpha_0 + alpha_1 * (y[t-1] - np.mean(y))**2
    
    # Scaling for numerical stability
    scaled_variances = conditional_variances / np.max(conditional_variances)
    
    return scaled_variances

# Assuming y is your time series data and initial parameters are omega, alpha, beta

def forward_pass_garch11(y, omega, alpha, beta):
    n = len(y)
    # Initialize arrays
    conditional_variances = np.zeros(n)
    conditional_variances[0] = np.var(y)  # Initial estimate
    
    # Forward pass: Calculate conditional variances
    for t in range(1, n):
        conditional_variances[t] = omega + alpha * (y[t-1] - np.mean(y[:t]))**2 + beta * conditional_variances[t-1]
    
    # Scaling for numerical stability (example)
    scaled_variances = conditional_variances / np.max(conditional_variances)
    
    return scaled_variances


# Correctly defining the forward pass functions for ARCH(1) and AR(1) models

def forward_pass_arch1(y, alpha_0, alpha_1):
    n = len(y)
    conditional_variances = np.zeros(n)
    conditional_variances[0] = np.var(y)  # Initial estimate
    for t in range(1, n):
        conditional_variances[t] = alpha_0 + alpha_1 * (y[t-1] - np.mean(y))**2
    scaled_variances = conditional_variances / np.max(conditional_variances)
    return scaled_variances

def forward_pass_ar1(y, phi_0, phi_1):
    n = len(y)
    predictions = np.zeros(n)
    for t in range(1, n):
        predictions[t] = phi_0 + phi_1 * y[t-1]
    scaled_predictions = (predictions - np.mean(predictions)) / np.std(predictions)
    return scaled_predictions
def forward_pass_garch11(y, omega, alpha, beta):
    n = len(y)
    conditional_variances = np.zeros(n)
    conditional_variances[0] = np.var(y)  # Initial estimate
    for t in range(1, n):
        conditional_variances[t] = omega + alpha * (y[t-1] - np.mean(y))**2 + beta * conditional_variances[t-1]
    scaled_variances = conditional_variances / np.max(conditional_variances)
    return scaled_variances

# Timed execution for GARCH(1,1)
start_time = time.time()
scaled_variances_garch11 = forward_pass_garch11(y_garch11, omega_garch11, alpha_garch11, beta_garch11)
time_garch11 = time.time() - start_time

# Timed execution for ARCH(1)
start_time = time.time()
scaled_variances_arch1 = forward_pass_arch1(y_arch1, alpha_0_arch1, alpha_1_arch1)
time_arch1 = time.time() - start_time

# Timed execution for AR(1)
start_time = time.time()
scaled_predictions_ar1 = forward_pass_ar1(y_ar1, phi_0_ar1, phi_1_ar1)
time_ar1 = time.time() - start_time

time_ar1, time_arch1, time_garch11


(0.2403244972229004, 390.55439949035645, 388.55611276626587)

# Backward Pass

In [5]:


# AR(1) Scaled Backward Pass
def scaled_backward_pass_ar1(y, phi_0, phi_1):
    n = len(y)
    residuals = np.zeros(n)
    for t in range(n - 2, -1, -1):  # Iterate backwards
        residuals[t] = y[t] - (phi_0 + phi_1 * y[t + 1])
    scaled_residuals = residuals / np.max(np.abs(residuals))
    return scaled_residuals

# ARCH(1) Scaled Backward Pass
def scaled_backward_pass_arch1(y, alpha_0, alpha_1):
    n = len(y)
    conditional_variances = np.zeros(n)
    conditional_variances[-1] = alpha_0 / (1 - alpha_1)  # Starting estimate for the last element
    for t in range(n - 2, -1, -1):  # Iterate backwards
        conditional_variances[t] = alpha_0 + alpha_1 * (y[t] - np.mean(y))**2
    scaled_variances = conditional_variances / np.max(conditional_variances)
    return scaled_variances

# GARCH(1,1) Scaled Backward Pass
def scaled_backward_pass_garch11(y, omega, alpha, beta):
    n = len(y)
    conditional_variances = np.zeros(n)
    conditional_variances[-1] = omega / (1 - alpha - beta)  # Starting estimate for the last element
    for t in range(n - 2, -1, -1):  # Iterate backwards
        conditional_variances[t] = omega + alpha * (y[t] - np.mean(y))**2 + beta * conditional_variances[t + 1]
    scaled_variances = conditional_variances / np.max(conditional_variances)
    return scaled_variances



# Timing and running the backward pass for each model
start_time = time.time()
scaled_residuals_ar1 = scaled_backward_pass_ar1(y_ar1, phi_0=0.5, phi_1=0.8)
time_ar1 = time.time() - start_time

start_time = time.time()
scaled_variances_arch1 = scaled_backward_pass_arch1(y_arch1, alpha_0=0.2, alpha_1=0.5)
time_arch1 = time.time() - start_time

start_time = time.time()
scaled_variances_garch11 = scaled_backward_pass_garch11(y_garch11, omega=0.1, alpha=0.05, beta=0.9)
time_garch11 = time.time() - start_time

# Output the timing for each
print(f"AR(1) Backward Pass Time: {time_ar1}")
print(f"ARCH(1) Backward Pass Time: {time_arch1}")
print(f"GARCH(1,1) Backward Pass Time: {time_garch11}")


AR(1) Backward Pass Time: 0.3909339904785156
ARCH(1) Backward Pass Time: 351.88525557518005
GARCH(1,1) Backward Pass Time: 383.9906690120697


In [6]:
time_2 = time.time()

print(time_2 - time_1)

1519.602458000183


100k observations 13.884862899780273 seconds
1mil observations

In [10]:
import numpy as np

def T_to_Gamma(T):
    """
    Transform T (working parameters) to Gamma (natural parameters).
    
    Parameters:
    - T: numpy array of shape (m, m) representing the matrix of working parameters.
    
    Returns:
    - Gamma: numpy array of shape (m, m) representing the matrix of natural parameters.
    """
    exp_T = np.exp(T)
    sum_exp_T = np.sum(exp_T, axis=1, keepdims=True) - np.diag(exp_T).reshape(-1, 1)
    Gamma = exp_T / (1 + sum_exp_T)
    np.fill_diagonal(Gamma, 1 - Gamma.sum(axis=1) + np.diag(Gamma))  # Adjust diagonal to ensure rows sum to 1
    return Gamma


def Gamma_to_T(Gamma):
    """
    Correctly transform Gamma (natural parameters) to T (working parameters),
    ensuring numerical stability and avoiding division by zero or negative numbers.
    
    Parameters:
    - Gamma: numpy array of shape (m, m) representing the matrix of natural parameters.
    
    Returns:
    - T: numpy array of shape (m, m) representing the matrix of working parameters.
    """
    # Subtract row sum (excluding diagonal) from 1 to get diagonal elements
    row_sums_excl_diag = Gamma.sum(axis=1) - np.diag(Gamma)
    diag_elements = 1 - row_sums_excl_diag
    T = np.log(Gamma / diag_elements[:, None])
    np.fill_diagonal(T, 0)  # Ensure diagonal elements are 0
    
    return T


m = 5  # Size of the matrix
T_example = np.random.rand(m, m)
np.fill_diagonal(T_example, 0)  # Ensure diagonal elements are 0

Gamma_transformed = T_to_Gamma(T_example)
T_transformed_back = Gamma_to_T(Gamma_transformed)

T_example, Gamma_transformed, T_transformed_back


(array([[0.        , 0.0398544 , 0.32017995, 0.18801481, 0.35462698],
        [0.11502064, 0.        , 0.67406007, 0.59703711, 0.46125526],
        [0.35606545, 0.79527672, 0.        , 0.48884447, 0.66941539],
        [0.03979099, 0.21561173, 0.15655972, 0.        , 0.22958734],
        [0.14381336, 0.45482681, 0.26770714, 0.90627292, 0.        ]]),
 array([[0.16527464, 0.17199458, 0.22764526, 0.19946193, 0.23562359],
        [0.14984843, 0.13356706, 0.26208365, 0.24265502, 0.21184585],
        [0.17355363, 0.26926552, 0.12156161, 0.19819784, 0.2374214 ],
        [0.1822796 , 0.21731817, 0.20485666, 0.17516892, 0.22037665],
        [0.15369732, 0.20976726, 0.17396932, 0.32945664, 0.13310946]]),
 array([[0.        , 0.0398544 , 0.32017995, 0.18801481, 0.35462698],
        [0.11502064, 0.        , 0.67406007, 0.59703711, 0.46125526],
        [0.35606545, 0.79527672, 0.        , 0.48884447, 0.66941539],
        [0.03979099, 0.21561173, 0.15655972, 0.        , 0.22958734],
        [0.14381

In [28]:
def extract_off_diagonal_params(M):
    """
    Extract off-diagonal parameters from a matrix.
    
    Parameters:
    - M: numpy array of shape (m, m)
    
    Returns:
    - params: List of off-diagonal elements of M.
    """
    m = M.shape[0]
    params = [M[i, j] for i in range(m) for j in range(m) if i != j]
    return params

def construct_matrix_from_params(params, m):
    """
    Construct a matrix from a list of off-diagonal parameters.
    
    Parameters:
    - params: List of off-diagonal elements.
    - m: Size of the matrix (m x m)
    
    Returns:
    - M: numpy array of shape (m, m) with off-diagonal elements filled from params.
    """
    if len(params) != m * (m - 1):
        raise ValueError("Number of parameters does not match the expected number for an m x m matrix.")
    
    M = np.zeros((m, m))
    param_index = 0
    for i in range(m):
        for j in range(m):
            if i != j:
                M[i, j] = params[param_index]
                param_index += 1
    return M

# Example usage:
m = 5# Matrix size
T_example = np.random.rand(m, m)
np.fill_diagonal(T_example, 0)  # Ensure diagonal elements are 0 for the example

# Extract parameters and then reconstruct the matrix
params = extract_off_diagonal_params(T_example)
print(len(params))
T_reconstructed = construct_matrix_from_params(params, m)
a = np.array((0.01,0.03, 0.04, 0.05,0.01,0.03, 0.04, 0.05,0.02,0.001,0.004,0.002,0.01, 0.02, 0.12,0.02,0.001,0.004,0.002,0.01))
b = construct_matrix_from_params(a, m)
print(b)
c = T_to_Gamma(b)
print(c)
print(np.sum(c, axis=1))
T_example, T_reconstructed


20
[[0.    0.01  0.03  0.04  0.05 ]
 [0.01  0.    0.03  0.04  0.05 ]
 [0.02  0.001 0.    0.004 0.002]
 [0.01  0.02  0.12  0.    0.02 ]
 [0.001 0.004 0.002 0.01  0.   ]]
[[0.19483354 0.19679165 0.2007671  0.20278485 0.20482287]
 [0.19679165 0.19483354 0.2007671  0.20278485 0.20482287]
 [0.20293582 0.19911643 0.19891742 0.19971468 0.19931565]
 [0.19506759 0.19702805 0.21774967 0.19312663 0.19702805]
 [0.19951931 0.20011877 0.19971893 0.20132309 0.19931989]]
[1. 1. 1. 1. 1.]


(array([[0.        , 0.87392922, 0.90834636, 0.85999983, 0.98355135],
        [0.54884945, 0.        , 0.46329687, 0.45816368, 0.57391113],
        [0.29435199, 0.36437901, 0.        , 0.56312429, 0.66468966],
        [0.77481207, 0.51090214, 0.07032278, 0.        , 0.82493266],
        [0.76263927, 0.84931322, 0.51039425, 0.86080962, 0.        ]]),
 array([[0.        , 0.87392922, 0.90834636, 0.85999983, 0.98355135],
        [0.54884945, 0.        , 0.46329687, 0.45816368, 0.57391113],
        [0.29435199, 0.36437901, 0.        , 0.56312429, 0.66468966],
        [0.77481207, 0.51090214, 0.07032278, 0.        , 0.82493266],
        [0.76263927, 0.84931322, 0.51039425, 0.86080962, 0.        ]]))

[[ 0. 10. 55.]
 [39.  0. 32.]
 [14. 13.  0.]]


[[1.29958143e-24 2.86251858e-20 1.00000000e+00]
 [9.99088949e-01 1.22560006e-16 9.11051194e-04]
 [7.31058134e-01 2.68941258e-01 6.07895834e-07]]
[1. 1. 1.]
